### Importing Data and Required Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# modelling 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold,cross_val_score,GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [2]:
df = pd.read_csv(r"C:\Users\kanis\Desktop\Breast-Cancer-Prediction\notebook\raw.csv")

In [3]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [4]:
df.drop("id",axis = 1 ,inplace=True)

In [5]:
df = df.replace({"M":1,"B":0})

C:\Users\kanis\AppData\Local\Temp\ipykernel_3788\1200107492.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"M":1,"B":0})


### Seperating out Features and label

In [6]:
y = df["diagnosis"]
x = df.drop("diagnosis",axis=1)

### Train-Test Split

In [7]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

### Feature Scaling

In [8]:
ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.fit_transform(x_test)

### Applying GridSearchCV

In [9]:
models = {
    "LogisticRegression":{
        "model_name":LogisticRegression(),
        "parameters":{
          "C":[0.1,1,10],
          "penalty":["l1","l2"],
          "solver":["liblinear","saga"]
    }
},
    "RandomForest":{
        "model_name":RandomForestClassifier(random_state = 42),
        "parameters":{
          "n_estimators": [50, 100, 200],
          "max_depth": [None, 10, 20],
          "min_samples_split": [2, 5, 10]
    }
},  
    "svm":{
        "model_name":SVC(),
        "parameters":{
          "C":[0.01,0.1,1],
          "gamma":[0.01,0.001,1,10],
          "kernel":["rbf","linear"]
    }
}
    }

In [10]:
best_models = {}
for i in models:
    clf= GridSearchCV(
        models[i]["model_name"],
        models[i]["parameters"],cv=5,n_jobs=-1)
    clf.fit(x_train,y_train)
    best_models[i] = clf.best_estimator_
    print(f"Best parameters for {i} : {clf.best_params_}")


Best parameters for LogisticRegression : {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
Best parameters for RandomForest : {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 200}
Best parameters for svm : {'C': 0.1, 'gamma': 0.01, 'kernel': 'linear'}


### Finding out Metrics for each Model

In [11]:
for model_name,model in best_models.items():
    y_pred = model.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    print("\n"f"{model_name} performance on Test set:")
    print(f"Accuracy: {accuracy}")
    print(f"precision: {precision}")
    print(f"recall: {recall}")
    print(f"f1: {f1}")



LogisticRegression performance on Test set:
Accuracy: 0.9824561403508771
precision: 0.9829367940398942
recall: 0.9824561403508771
f1: 0.9823691172375383

RandomForest performance on Test set:
Accuracy: 0.9649122807017544
precision: 0.9652053622194477
recall: 0.9649122807017544
f1: 0.9647382344750765

svm performance on Test set:
Accuracy: 0.9824561403508771
precision: 0.9829367940398942
recall: 0.9824561403508771
f1: 0.9823691172375383


From the above analysis it can be deduced that the model which is giving good scores is LogisticRegression